# Contents
1. Installation of publicly available programs used
    1. Set up channels
    2. Create an environment and install the packages
2. Alignment
    1. HISAT 
    2. Samtools
    3. MultiQC
3. ChIP-Seq
    1. MACS
    2. Bedtools
4. RNA-Seq
    1. HTSeq
    2. featureCounts
    3. DESeq
    4. Salmon
    5. dupRadar

# Installation of publicly available programs used
This jupyter notebook contains all of the code we need to run an RNA-Seq and ChIP-Seq pipeline, but it uses some publicly available software to take care of data acquisition, analysis, and visualization.

To install this software, we are going to use a package manager that allows us to install software with a simple command.  We use Bioconda (https://bioconda.github.io/), which is a channel based on the conda package manager that specializes in bioinformatics packages. In conda, a channel contains a set of software managed by the same group.  Since we are using bioinformatics tools, we are using the Bioconda <i>channel</i> of the conda <i>package manager</i>.  There are over 3000 packages available through the Bioconda channel: https://bioconda.github.io/recipes.html#recipes.

## Set up channels
You will need to add the bioconda channel as well as the other channels bioconda depends on. It is important to add them in the specific order below so that the priority is set correctly (that is, bioconda is highest priority).

The conda-forge channel contains many general-purpose packages not already found in the defaults channel. The r channel is only included due to backward compatibility. It is not mandatory, but without the r channel, packages compiled against R 3.3.1 might not work.

Note: This tutorial uses cells written in python and unix to perform its analyses. Lines that are written in unix are prefixed by an exclamation point. 

Select the following cell and run it. To run a cell, select the cell, click "Cell" the upper taskbar, and select "Run Cells". Or click the cell and press shift + enter.

Alternatively, the contents of any cell may be copy+pasted into the terminal emulator to run.

In [ ]:
!conda config --add channels defaults
!conda config --add channels conda-forge
!conda config --add channels bioconda

## Create an environment and install the packages

In this tutorial we will create an environment named "tutorial" and install the packages in there. Environments offer ways of installing packages in specific environments so they can be managed and run for different specifications. You can create, export, list, remove and update environments that have different versions of Python and/or packages installed in them. Switching or moving between environments is called activating the environment. You can also share an environment file.

This command will create an environment "tutorial" in which to install the packages used in this tutorial.

Run the following commands to create the environment. When it asks if you want to proceed, type "y" and press enter.

In [ ]:
!conda create -n tutorial hisat2 multiqc macs2 salmon bioconductor-deseq matplotlib ggplot pybedtools samtools bioconductor-rsamtools bedtools htseq subread bioconductor-rsubread

Then activate the environment with the following command.

In [ ]:
# For Mac and Linux

!source activate tutorial

# For Windows

!activate tutorial

# Alignment
## HISAT (Hierarchical Indexing for Spliced Alignment of Transcripts)

In this tutorial, we'll use Hisat to align the sample reads to a reference genome. Hisat automatically downloads and preprocesses the reads so they're ready to be aligned. Hisat (hierarchical indexing for spliced alignment of transcripts) is a highly efficient system for aligning reads from RNA sequencing experiments. HISAT uses an indexing scheme based on the Burrows-Wheeler transform and the Ferragina-Manzini (FM) index, employing two types of indexes for alignment: a whole-genome FM index to anchor each alignment and numerous local FM indexes for very rapid extensions of these alignments. HISAT’s hierarchical index for the human genome contains 48,000 local FM indexes, each representing a genomic region of ~64,000 bp.

The RNA-Seq data we'll use is from https://www.ncbi.nlm.nih.gov/Traces/study/?acc=SRP106028 and the ChIP-Seq data is from https://www.ncbi.nlm.nih.gov/Traces/study/?acc=SRP132584

The model organism for this project is Yeast i.e. Saccharomyces cerevisiae. For RNA-Seq, yeast data between euploid and aneuoploid conditions will be compared. For ChIP-SEq, yeast data between 3AT-treated and untreated conditions will be compared.

In [3]:
from pandas import read_csv

RNASeqSRARunTableFile='data/RNASeqSRA.tsv'
RNASeqSRATable = read_csv(RNASeqSRARunTableFile, delimiter='\t')
RNASeqoutrun = (RNASeqSRATable["Run"]).astype(list)
RNASeqoutputSam = "test/" + RNASeqoutrun + ".sam"
RNASeqoutputAlignmentSummary = "test/" + RNASeqoutrun + ".txt"
RNASeqoutputMetrics = "test/" + RNASeqoutrun + ".metrics"
RNASeqoutputSortBam = "test/" + RNASeqoutrun + ".sorted.bam"

ChIPSeqSRARunTableFile='data/ChIPSeqSRA.tsv'
ChIPSeqSRATable = read_csv(ChIPSeqSRARunTableFile, delimiter='\t')
ChIPSeqoutrun = (ChIPSeqSRATable["Run"]).astype(list)
ChIPSeqoutputSam = "test/" + ChIPSeqoutrun + ".sam"
ChIPSeqoutputAlignmentSummary = "test/" + ChIPSeqoutrun + ".txt"
ChIPSeqoutputMetrics = "test/" + ChIPSeqoutrun + ".metrics"
ChIPSeqoutputSortBam = "test/" + ChIPSeqoutrun + ".sorted.bam"

In [ ]:
Then run the following command to create the yeast index.

In [ ]:
!bash scripts/make_yeast_index.sh

In [ ]:
Align the RNA-Seq samples using Hisat. 

In [ ]:
for index, individual in enumerate(RNASeqoutrun):
    run = RNASeqoutrun[index]
    summary = RNASeqoutputAlignmentSummary[index] 
    metrics = RNASeqoutputMetrics[index]
    sam = RNASeqoutputSam[index]
    bam = RNASeqoutputSortBam[index]
    !hisat2 -x yeast_index/genome --sra-acc $run --new-summary --summary-file $summary --met-file $metrics -S $sam

## Samtools 

We'll use samtools to sort the output files and convert them to bam files.

Sort the output files and convert them to bam files.

In [ ]:
for index, individual in enumerate(RNASeqoutrun):
    run = RNASeqoutrun[index]
    summary = RNASeqoutputAlignmentSummary[index] 
    metrics = RNASeqoutputMetrics[index]
    sam = RNASeqoutputSam[index]
    bam = RNASeqoutputSortBam[index]
    !samtools view -bSF4 $sam | samtools sort -o $bam
    

In [ ]:
Do the same thing for ChIP-Seq samples.

In [4]:
for index, individual in enumerate(ChIPSeqoutrun):
    run = ChIPSeqoutrun[index]
    summary = ChIPSeqoutputAlignmentSummary[index] 
    metrics = ChIPSeqoutputMetrics[index]
    sam = ChIPSeqoutputSam[index]
    bam = ChIPSeqoutputSortBam[index]
    index = "yeast_index/genome"
    !hisat2 -x $index --sra-acc $run --new-summary --summary-file $summary --met-file $metrics -S $sam

Error: Could not open alignment output file ../../../test/SRR6703656.sam
Error: Encountered internal HISAT2 exception (#1)
Command: /home/ubuntu/miniconda3/envs/tutorial/bin/hisat2-align-s --wrapper basic-0 -x yeast_index/genome --sra-acc SRR6703656 --new-summary --summary-file ../.test/SRR6703656.txt --met-file ../../../test/SRR6703656.metrics -S ../../../test/SRR6703656.sam 
(ERR): hisat2-align exited with value 1
Error: Could not open alignment output file ../../../test/SRR6703661.sam
Error: Encountered internal HISAT2 exception (#1)
Command: /home/ubuntu/miniconda3/envs/tutorial/bin/hisat2-align-s --wrapper basic-0 -x yeast_index/genome --sra-acc SRR6703661 --new-summary --summary-file ../.test/SRR6703661.txt --met-file ../../../test/SRR6703661.metrics -S ../../../test/SRR6703661.sam 
(ERR): hisat2-align exited with value 1
Error: Could not open alignment output file ../../../test/SRR6703662.sam
Error: Encountered internal HISAT2 exception (#1)
Command: /home/ubuntu/miniconda3/envs

In [ ]:
for index, individual in enumerate(ChIPSeqoutrun):
    run = ChIPSeqoutrun[index]
    summary = ChIPSeqoutputAlignmentSummary[index] 
    metrics = ChIPSeqoutputMetrics[index]
    sam = ChIPSeqoutputSam[index]
    bam = ChIPSeqoutputSortBam[index]
    !samtools view -bSF4 $sam | samtools sort -o $bam

## MultiQC

This section details quality control checks on the read data from either RNAseq or ChIPseq data using MultiQC. MultiQC takes all output and log files from an alignment software program and aggregates the information from all samples into one convenient report (html by default).

MultiQC was installed earlier in the tutorial, so all we need to do is run it on the data.

MultiQC is configured to run the same no matter what type of sequencing data is available, therefore the same command can be used to analyze either our RNAseq data or our ChIPseq data.  We include the option 'hisat_output' since we are aligning using the HISAT2 program.  See http://multiqc.info/docs/ for more information.

We use the 'hisat_output' option because we are analyzing data downloaded and aligned using the HISAT2 program.  We use the '--force' option to overwrite any previous versions of the multiqc_report.  '--quiet' only shows log warnings.

In [ ]:
!multiqc "".join(RNASeqoutrun) --quiet --outdir test/multiqc_rnaseq --force
!multiqc "".join(ChIPSeqoutrun) --quiet --outdir test/multiqc_chipseq --force

# ChIP-Seq
## MACS (Model-based Analysis for ChIP-Seq)

Peak-calling is one of the main steps scientists use in determining the locations where protein is bound in DNA. Peak detection software, such as MACS (Model-Based Analysis for ChIP-Seq), call peaks using the aligned sequecnes as input and returns precise locations of predicted peaks as output. In this tutorial, we'll use MACS.

More information about MACS: http://liulab.dfci.harvard.edu/MACS/Download.html

In [5]:
ChIPSeqControl = ChIPSeqSRATable.loc[ChIPSeqSRATable["source_name"] == "Untreated"]["Run"].astype(list)
ChIPSeqTreatment = ChIPSeqSRATable.loc[ChIPSeqSRATable["source_name"] != "Untreated"]["Run"].astype(list)

print("Control sample " + ChIPSeqControl)
print("Treatment sample " + ChIPSeqTreatment)

0    Control sample SRR6703656
2    Control sample SRR6703662
Name: Run, dtype: object
1    Treatment sample SRR6703661
3    Treatment sample SRR6703663
Name: Run, dtype: object


In [6]:
for index, individual in enumerate(ChIPSeqControl):
    outputdirectory = "test/" + ChIPSeqTreatment.iloc[index]
    immunoprecipitate = "test/" + ChIPSeqTreatment.iloc[index] + ".sorted.bam"
    sample = "test/" + ChIPSeqControl.iloc[index] + ".sorted.bam"
    !macs2 callpeak -c $sample -t $immunoprecipitate -n $immunoprecipitate --outdir outputdirectory

INFO  @ Wed, 18 Apr 2018 15:03:36: 
# Command line: callpeak -c test/SRR6703656.sorted.bam -t test/SRR6703661.sorted.bam -n test/SRR6703661.sorted.bam --outdir test/SRR6703661.sorted.bam
# ARGUMENTS LIST:
# name = test/SRR6703661.sorted.bam
# format = AUTO
# ChIP-seq file = ['test/SRR6703661.sorted.bam']
# control file = ['test/SRR6703656.sorted.bam']
# effective genome size = 2.70e+09
# band width = 300
# model fold = [5, 50]
# qvalue cutoff = 5.00e-02
# Larger dataset will be scaled towards smaller dataset.
# Range for calculating regional lambda is: 1000 bps and 10000 bps
# Broad region calling is off
# Paired-End mode is off
 
INFO  @ Wed, 18 Apr 2018 15:03:36: #1 read tag files... 
INFO  @ Wed, 18 Apr 2018 15:03:36: #1 read treatment tags... 
INFO  @ Wed, 18 Apr 2018 15:03:36: Detected format is: BAM 
INFO  @ Wed, 18 Apr 2018 15:03:36: * Input file is gzipped. 
INFO  @ Wed, 18 Apr 2018 15:03:40:  1000000 
INFO  @ Wed, 18 Apr 2018 15:03:44:  2000000 
INFO  @ Wed, 18 Apr 2018 15:03:

INFO  @ Wed, 18 Apr 2018 15:13:32:  7000000 
INFO  @ Wed, 18 Apr 2018 15:13:36:  8000000 
INFO  @ Wed, 18 Apr 2018 15:13:39:  9000000 
INFO  @ Wed, 18 Apr 2018 15:13:43:  10000000 
INFO  @ Wed, 18 Apr 2018 15:13:47:  11000000 
INFO  @ Wed, 18 Apr 2018 15:13:51:  12000000 
INFO  @ Wed, 18 Apr 2018 15:13:55:  13000000 
INFO  @ Wed, 18 Apr 2018 15:13:59:  14000000 
INFO  @ Wed, 18 Apr 2018 15:14:03:  15000000 
INFO  @ Wed, 18 Apr 2018 15:14:07:  16000000 
INFO  @ Wed, 18 Apr 2018 15:14:11:  17000000 
INFO  @ Wed, 18 Apr 2018 15:14:15:  18000000 
INFO  @ Wed, 18 Apr 2018 15:14:19:  19000000 
INFO  @ Wed, 18 Apr 2018 15:14:22:  20000000 
INFO  @ Wed, 18 Apr 2018 15:14:26:  21000000 
INFO  @ Wed, 18 Apr 2018 15:14:30:  22000000 
INFO  @ Wed, 18 Apr 2018 15:14:34:  23000000 
INFO  @ Wed, 18 Apr 2018 15:14:38:  24000000 
INFO  @ Wed, 18 Apr 2018 15:14:42:  25000000 
INFO  @ Wed, 18 Apr 2018 15:14:46:  26000000 
INFO  @ Wed, 18 Apr 2018 15:14:50:  27000000 
INFO  @ Wed, 18 Apr 2018 15:14:54:  2

INFO  @ Wed, 18 Apr 2018 15:22:23: 
# Command line: callpeak -c test/SRR6703662.sorted.bam -t test/SRR6703663.sorted.bam -n test/SRR6703663.sorted.bam --outdir test/SRR6703663.sorted.bam
# ARGUMENTS LIST:
# name = test/SRR6703663.sorted.bam
# format = AUTO
# ChIP-seq file = ['test/SRR6703663.sorted.bam']
# control file = ['test/SRR6703662.sorted.bam']
# effective genome size = 2.70e+09
# band width = 300
# model fold = [5, 50]
# qvalue cutoff = 5.00e-02
# Larger dataset will be scaled towards smaller dataset.
# Range for calculating regional lambda is: 1000 bps and 10000 bps
# Broad region calling is off
# Paired-End mode is off
 
INFO  @ Wed, 18 Apr 2018 15:22:23: #1 read tag files... 
INFO  @ Wed, 18 Apr 2018 15:22:23: #1 read treatment tags... 
INFO  @ Wed, 18 Apr 2018 15:22:23: Detected format is: BAM 
INFO  @ Wed, 18 Apr 2018 15:22:23: * Input file is gzipped. 
INFO  @ Wed, 18 Apr 2018 15:22:27:  1000000 
INFO  @ Wed, 18 Apr 2018 15:22:32:  2000000 
INFO  @ Wed, 18 Apr 2018 15:22:

INFO  @ Wed, 18 Apr 2018 15:31:46:  159000000 
INFO  @ Wed, 18 Apr 2018 15:31:49:  160000000 
INFO  @ Wed, 18 Apr 2018 15:31:53:  161000000 
INFO  @ Wed, 18 Apr 2018 15:31:56:  162000000 
INFO  @ Wed, 18 Apr 2018 15:32:00:  163000000 
INFO  @ Wed, 18 Apr 2018 15:32:03:  164000000 
INFO  @ Wed, 18 Apr 2018 15:32:07:  165000000 
INFO  @ Wed, 18 Apr 2018 15:32:10:  166000000 
INFO  @ Wed, 18 Apr 2018 15:32:14:  167000000 
INFO  @ Wed, 18 Apr 2018 15:32:17:  168000000 
INFO  @ Wed, 18 Apr 2018 15:32:21:  169000000 
INFO  @ Wed, 18 Apr 2018 15:32:24:  170000000 
INFO  @ Wed, 18 Apr 2018 15:32:28:  171000000 
INFO  @ Wed, 18 Apr 2018 15:32:31:  172000000 
INFO  @ Wed, 18 Apr 2018 15:32:35:  173000000 
INFO  @ Wed, 18 Apr 2018 15:32:38:  174000000 
INFO  @ Wed, 18 Apr 2018 15:32:42:  175000000 
INFO  @ Wed, 18 Apr 2018 15:32:45:  176000000 
INFO  @ Wed, 18 Apr 2018 15:32:49:  177000000 
INFO  @ Wed, 18 Apr 2018 15:32:52:  178000000 
INFO  @ Wed, 18 Apr 2018 15:32:56:  179000000 
INFO  @ Wed, 

INFO  @ Wed, 18 Apr 2018 15:42:07:  96000000 
INFO  @ Wed, 18 Apr 2018 15:42:11:  97000000 
INFO  @ Wed, 18 Apr 2018 15:42:15:  98000000 
INFO  @ Wed, 18 Apr 2018 15:42:19:  99000000 
INFO  @ Wed, 18 Apr 2018 15:42:23:  100000000 
INFO  @ Wed, 18 Apr 2018 15:42:26:  101000000 
INFO  @ Wed, 18 Apr 2018 15:42:30:  102000000 
INFO  @ Wed, 18 Apr 2018 15:42:34:  103000000 
INFO  @ Wed, 18 Apr 2018 15:42:37:  104000000 
INFO  @ Wed, 18 Apr 2018 15:42:41:  105000000 
INFO  @ Wed, 18 Apr 2018 15:42:44:  106000000 
INFO  @ Wed, 18 Apr 2018 15:42:48:  107000000 
INFO  @ Wed, 18 Apr 2018 15:42:51:  108000000 
INFO  @ Wed, 18 Apr 2018 15:42:55:  109000000 
INFO  @ Wed, 18 Apr 2018 15:42:59:  110000000 
INFO  @ Wed, 18 Apr 2018 15:43:03:  111000000 
INFO  @ Wed, 18 Apr 2018 15:43:08:  112000000 
INFO  @ Wed, 18 Apr 2018 15:43:11:  113000000 
INFO  @ Wed, 18 Apr 2018 15:43:14:  114000000 
INFO  @ Wed, 18 Apr 2018 15:43:18:  115000000 
INFO  @ Wed, 18 Apr 2018 15:43:21:  116000000 
INFO  @ Wed, 18 A

For an in-depth discussion of what MACS2 does: https://github.com/taoliu/MACS/wiki/Advanced:-Call-peaks-using-MACS2-subcommands

## Bedtools

In this tutorial, we'll use Bedtools to extract the intersecting regions of the MACS output between the experimental conditions.

The Bedtools suite of programs is widely used for genomic interval manipulation or "genome algebra". pybedtools wraps and extends Bedtools and offers feature-level manipulations from within Python.

First we'll sort the output. The following line uses the `sort` command to sort the MACS output.

In [ ]:
!sort -k 1,1 -k2,2n test/SRR6703661/SRR6703661.narrowPeak > test/SRR6703661/SRR6703661.narrowPeak.sorted
!sort -k 1,1 -k2,2n test/SRR6703663/SRR6703663.narrowPeak > test/SRR6703661/SRR6703663.narrowPeak.sorted

Then we'll use pybedtools to truncate the output files to chromosome so no ends are out of bounds. 

In [ ]:
from pybedtools import BedTool

BedTool(test/SRR6703661/SRR6703661.narrowPeak.sorted).truncate_to_chrom('sacCer3').saveas(test/SRR6703661/SRR6703661.narrowPeak.sorted.truncated)
BedTool(test/SRR6703661/SRR6703663.narrowPeak.sorted).truncate_to_chrom('sacCer3').saveas(test/SRR6703661/SRR6703663.narrowPeak.sorted.truncated)

Then we'll find the intersecting regions between the experimental conditions and the control conditions. 

In [ ]:
!bedtools intersect -a test/SRR6703661/SRR6703661.narrowPeak.sorted.truncated -b test/SRR6703661/SRR6703663.narrowPeak.sorted.truncated -u > test/ChIPSeqintersect.bed

# RNA-Seq
## HTSeq (High-through sequencing)

HTSeq is a Python library to facilitate the rapid development of RNA-Seq analysis. HTSeq offers parsers for many common data formats in HTS projects, as well as classes to represent data, such as genomic coordinates, sequences, sequencing reads, alignments, gene model information and variant calls, and provides data structures that allow for querying via genomic coordinates. In this tutorial we will use htseq-count, a tool developed with HTSeq that preprocesses RNA-Seq data for differential expression analysis by counting the overlap of reads with genes.

In [ ]:
from pandas import read_csv

gtf = "data/Saccharomyces_cerevisiae.R64-1-1.84.gtf"

RNASeqSRARunTableFile='data/RNASeqSRA.tsv'
RNASeqSRATable = read_csv(RNASeqSRARunTableFile, delimiter='\t')
RNASeqoutrun = (RNASeqSRATable["Run"]).astype(list)
RNASeqoutputSortBam = "test/" + RNASeqoutrun + ".sorted.bam"

In [ ]:
for index, individual in enumerate(RNASeqoutputSortBam):
    input = individual
    output = individual + ".genecount.txt"
    !htseq-count -m intersection-nonempty -s no -f bam $input $gtf > $output

## featureCounts

featureCounts is a highly efficient general-purpose read summarization program that counts mapped reads for genomic features such as genes, exons, promoter, gene bodies, genomic bins and chromosomal locations. It can be used to count both RNA-seq and genomic DNA-seq reads. It is available in the SourceForge Subread package or the Bioconductor Rsubread package.

In [ ]:
for index, individual in enumerate(RNASeqoutputSortBam):
    input = individual
    !featureCounts -s -a $gtf -o test/featureCounts $input

## DESeq (Differential Expression Sequencing)

Estimate variance-mean dependence in count data from high-throughput sequencing assays and test for differential expression based on a model using the negative binomial distribution

In [ ]:
!Rscript scripts/runDeseq.R

## Salmon

Salmon is a tool for quantifying the expression of transcripts using RNA-seq data. Salmon uses new algorithms (specifically, coupling the concept of quasi-mapping with a two-phase inference procedure) to provide accurate expression estimates very quickly (i.e. wicked-fast) and while using little memory. Salmon performs its inference using an expressive and realistic model of RNA-seq data that takes into account experimental attributes and biases commonly observed in real RNA-seq data.

In [ ]:
!wget -O data/transcript.fa.gz ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_rna.fna.gz

In [ ]:
Then unzip the transcript.fa.gz file.

In [ ]:
!gunzip data/transcript.fa.gz

In [ ]:
Finally, create the salmon index and run salmon on it.

In [ ]:
salmon_index = "test/yeast-salmon.idx"

!salmon index -t data/transcript.fa -i $salmon_index --type quasi -k 31


In [ ]:
from pandas import read_csv

RNASeqSRARunTableFile='data/RNASeqSRA.tsv'
RNASeqSRATable = read_csv(RNASeqSRARunTableFile, delimiter='\t')
RNASeqoutrun = (RNASeqSRATable["Run"])
RNASeqBam = "test/" + RNASeqoutrun + ".sorted.bam"

for index, individual in enumerate(RNASeqBam):
    !salmon quant --targets $salmon_index -l A --output test -a $individual

## dupRadar
dupRadar provides an easy way to distinguish between artifactual vs natural duplicate reads in RNA-Seq data. Prior to dupRadar only global duplication rates were used and they don't take into account the effect of gene expression levels. dupRadar relates duplication rates and length normalized read counts of every gene to model the dependency of both variables.

In [ ]:
for index, individual in enumerate(RNASeqoutputSortBam):
    !Rscript scripts/dupRadar.R $individual $gtf